# Physics Informed Neural Networks <br> F1 Car Front Wing Aerodymanics

## PINN

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from pinn import PINN

In [2]:
in_filepath = "/Users/ggito/repos/pinns/data/"
points_filename = "front_wing_points_final.csv"

wing_df = pd.read_csv(in_filepath + points_filename)

print(wing_df)

              x         y         z
0      0.440148  0.073950  0.103123
1      0.713695  0.209429  0.055195
2      0.451790  0.021569  0.052462
3      0.032607  0.154912  0.108069
4      0.750952  0.139930  0.113273
...         ...       ...       ...
19995  0.913177  0.282509  0.051195
19996  0.115440  0.221203  0.028832
19997  0.453917  0.034118  0.052462
19998  0.556022  0.063779  0.101915
19999  0.030382  0.130567  0.011402

[20000 rows x 3 columns]


In [3]:
x_min, y_min, z_min = wing_df.min()
x_max, y_max, z_max = wing_df.max()

overall_min = min(x_min, y_min, z_min)
overall_max = max(x_max, y_max, z_max)

range = (overall_min, overall_max)
print(range)

(0.0, 1.0)


In [4]:
import plotly.express as px

df = px.data.iris()
fig = px.scatter_3d(wing_df, x='x', y='y', z='z')
fig.update_traces(marker_size = 1)
fig.update_traces(marker_color = 'purple')
fig.update_layout(
    scene = dict(
        xaxis = dict(range=range),
        yaxis = dict(range=range),
        zaxis = dict(range=range)))

fig.show()

In [5]:
out_filepath = "/Users/ggito/repos/pinns/data/"
pinn = torch.load(out_filepath + 'pinn3.pt')
pinn.eval()

PINN(
  (layers): ModuleList(
    (0): Linear(in_features=4, out_features=1024, bias=True)
    (1-2): 2 x Linear(in_features=1024, out_features=1024, bias=True)
    (3): Linear(in_features=1024, out_features=4, bias=True)
  )
)

In [6]:
x_max = 1
y_max = 1
z_max = 1
t_max = 1

Nx = 10
Ny = 10
Nz = 10
Nt = 10

dx = x_max / (Nx - 1)
dy = y_max / (Ny - 1)
dz = z_max / (Nz - 1)
dt = t_max / (Nt - 1)

x_test = np.linspace(0, x_max, Nx)
y_test = np.linspace(0, y_max, Ny)
z_test = np.linspace(0, z_max, Nz)
t_test = np.full(Nt, 0.5)

x_grid, y_grid, z_grid = np.meshgrid(x_test, y_test, z_test)

In [7]:
t_test

array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5])

In [8]:
xyz_combinations = torch.cartesian_prod(torch.tensor(x_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.float32), torch.tensor(z_test, dtype=torch.float32))

In [9]:
num_of_points = xyz_combinations.shape[0]

In [10]:
t = torch.full((num_of_points, 1), 0.5)

In [11]:
xyzt_combinations = torch.cat((xyz_combinations, t), dim=1)

In [12]:
xyzt_combinations.shape

torch.Size([1000, 4])

In [13]:
if torch.backends.mps.is_available():
  device = torch.device("mps")
  x = torch.ones(1, device=device)
  print(x)
else:
  print("MPS device not found.")
  device = "cpu"

tensor([1.], device='mps:0')


In [14]:
input = xyzt_combinations.to(device)

In [15]:
output = pinn(input)

In [16]:
output.shape

torch.Size([1000, 4])

In [17]:
u_test = output[:, 0].to("cpu").detach().numpy()
v_test = output[:, 1].to("cpu").detach().numpy()
w_test = output[:, 2].to("cpu").detach().numpy()
p_test = output[:, 3].to("cpu").detach().numpy()

In [18]:
import plotly.graph_objects as go
import pandas as pd

scatter = go.Scatter3d(
    x=wing_df['x'],
    y=wing_df['y'],
    z=wing_df['z'],
    mode='markers',
    marker=dict(size=1, color='purple')
)

cone = go.Cone(
    x=x_grid.flatten(),
    y=y_grid.flatten(),
    z=z_grid.flatten(),
    u=u_test,
    v=v_test,
    w=w_test,
    sizeref=0.5,
    colorscale='haline',
    cmin=0,
    cmax=2
)

fig = go.Figure(data=[scatter, cone])

fig.update_layout(
    scene = dict(
        xaxis = dict(range=range),
        yaxis = dict(range=range),
        zaxis = dict(range=range)))

fig.show()